In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys


from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import os

from browsermobproxy import Server
import pandas as pd


In [2]:
options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

driver.get("https://longportapp.com/en/markets/")

print(driver.title)
driver.page_source

# driver.close()
driver.quit()

LongPort | Research and trade with high-value investors


In [3]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

import undetected_chromedriver as uc

# options = Options()
options = uc.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument("--auto-open-devtools-for-tabs")
driver = uc.Chrome(options=options)
# driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

driver.get("https://longportapp.com/en/markets/")
print(driver.title)
driver.quit()

LongPort | Research and trade with high-value investors


In [9]:
import os
import json
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

# Directory to save the downloaded files
download_dir = "/home/disk1/finance_quant/efinance/downloaded_files"
os.makedirs(download_dir, exist_ok=True)

# Initialize ChromeDriver
options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

prefs = {
    "download.default_directory": download_dir,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True
}
options.add_experimental_option("prefs", prefs)

# Enable logging
options.set_capability('goog:loggingPrefs', {'performance': 'ALL'})

driver = webdriver.Chrome(options=options)

# Function to save downloaded file (if applicable)
def save_file(url, content):
    filename = os.path.join(download_dir, url.split('/')[-1])
    with open(filename, 'wb') as f:
        f.write(content.encode('utf-8'))
    print(f"Downloaded {filename}")

# Function to process performance logs
def process_browser_logs_for_network_events(logs):
    for entry in logs:
        log = json.loads(entry['message'])['message']
        if (
            'Network.responseReceived' in log['method']
            and 'params' in log
            and 'response' in log['params']
        ):
            response = log['params']['response']
            url = response['url']
            if any(url.endswith(ext) for ext in ['.js', '.css', '.json']):
                try:
                    content = driver.execute_cdp_cmd('Network.getResponseBody', {'requestId': log['params']['requestId']})
                    save_file(url, content['body'])
                except Exception as e:
                    print(f"Error getting response body for {url}: {e}")

try:
    # Enable network interception
    driver.execute_cdp_cmd('Network.enable', {})

    # Open a webpage
    driver.get("https://longportapp.com/en/markets/")

    # Wait for page to load (adjust wait time and selector as needed)
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "portal")))

    # Add a sleep to allow time for resources to be captured
    import time
    time.sleep(5)

    # Retrieve and process performance logs
    logs = driver.get_log('performance')
    process_browser_logs_for_network_events(logs)

finally:
    # Quit the driver
    driver.quit()

Downloaded /home/disk1/finance_quant/efinance/downloaded_files/sensorsdata@1.16.10.min.js
Downloaded /home/disk1/finance_quant/efinance/downloaded_files/iconfont.css
Downloaded /home/disk1/finance_quant/efinance/downloaded_files/index.43ca75ce.js
Downloaded /home/disk1/finance_quant/efinance/downloaded_files/webpage.30ef6644.css
Downloaded /home/disk1/finance_quant/efinance/downloaded_files/chunk-element-plus.14de3114.js
Downloaded /home/disk1/finance_quant/efinance/downloaded_files/lang.a0ea826d.js
Downloaded /home/disk1/finance_quant/efinance/downloaded_files/build.0eb1f581.js
Downloaded /home/disk1/finance_quant/efinance/downloaded_files/tenant.50ee810f.js
Downloaded /home/disk1/finance_quant/efinance/downloaded_files/feedmap.95e5823c.js
Downloaded /home/disk1/finance_quant/efinance/downloaded_files/templates.e97d783a.js
Downloaded /home/disk1/finance_quant/efinance/downloaded_files/systemViews.a1893262.js
Downloaded /home/disk1/finance_quant/efinance/downloaded_files/font_2621450_x

In [2]:
# Start the BrowserMob Proxy server
import os

path_to_binary = "/home/disk1/tools/browsermob-proxy-2.1.4/bin/browsermob-proxy"
if not os.path.exists(path_to_binary):
    print("Error: Browsermob-Proxy binary not found at", path_to_binary)

server = Server(path_to_binary)
server.start()
proxy = server.create_proxy()

options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument(f"--proxy-server={proxy.proxy}")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# Start capturing network traffic
proxy.new_har("network_capture")

driver.get("https://longportapp.com/en/markets/")

# Perform actions as needed
# For demonstration, we will just wait for a few seconds
driver.implicitly_wait(10)

# Get the HAR data (HTTP Archive)
har_data = proxy.har

# Stop the proxy server and browser
driver.quit()
server.stop()

# Extract relevant data from HAR
network_data = []
for entry in har_data['log']['entries']:
    request = entry['request']
    response = entry['response']
    network_data.append({
        "Name": request['url'],
        "Status": response['status'],
        "Type": request['headers'][0]['value'] if request['headers'] else '',
        "Initiator": entry['pageref']
    })

# Create a DataFrame
df = pd.DataFrame(network_data)

# Display the DataFrame
display(df)


ProxyServerError: Can't connect to Browsermob-Proxy

driver.get_page

In [1]:
driver.quit()

NameError: name 'driver' is not defined